Name:   
Matrikelnummer:  
Email:   
   
Name:   
Matrikelnummer:   
Email:

Name:    
Matrikelnummer:    
Email:    

In [ ]:
# TODO: Import necessary libraries

## Important: We recommend that you use GPU(s) for these tasks. Google Colab is one such place where you could use GPU(s) for free for a certain duration.

# 10.2.1 Design your RNN (4 points)

Please create a ```solution.py``` file where you define the following:


1. A ```function``` where you use pytorch's Dataset and Dataloader class, and it should return you the desired split for the dataset. The function should have ```split``` as one of its argument and the call to Dataset class should respect this argument. You will manually need to download the dataset first. The desired role of function is as follows:
    - Use the ```Large Movie Review Dataset``` dataset. [Link](https://ai.stanford.edu/~amaas/data/sentiment)
    - Create Dataset object for different splits
    - Computers don't work with natural language, so we have to convert it to some sort of numbers. One such idea would be to use GloVe embeddings for the conversion. Depending on how you choose to do this, you might also have to take care of padding. **Note:** We encourage using the 300d GloVe embeddings.
    - Returns the Dataloader object for specified split
    - **(Optional)** Try one-hot encoding in-place of GloVe to see how big of an improvement GloVe was for embedding space. There are other (possible but not recommended) ways to do embeddings, such as get POS tags for each word or use a dictionary to define polarity for each word.
    
2. Multiple ```class``` for your implementation of your networks which does the following:
    - Define a RNN class with appropriate layers and hyperparameters
    - Define a LSTM class with appropriate layers and hyperparameters
    - **(Optionally)** Implement Bi-LSTM, Bi-RNN, Bi-GRU and do a comparison with the one-directional implementation


### Train your model(s) by importing your implementation from ```solution.py``` file in this notebook

In [ ]:
# Train your models

# 10.2.2 Transformers (3 points)

Let's fine-tune a Transformers based Model for the same task. We will use ```RoBERTa``` for the same. Note that for **converting text to some sort of numbers** you will need to use the tokenizer. HuggingFace has a nice introduction for the same [here](https://huggingface.co/course/chapter1/1)

In [ ]:
# Train your model